In [9]:
import operator
from collections import deque

In [1]:
datafile = 'data/18-1.txt'

In [2]:
with open(datafile) as fh:
    data = [x.strip() for x in fh]

In [10]:
class Calculator:

    def __init__(self):
        self.stack = []
        self.input = deque()
    
    def __call__(self, s):
        self.__init__()
        self.load(s)
        return self.run()
    
    def tokenize(self, s):
        cpbuf = []
        for t in s.split():
            while t.startswith('('):
                yield '('
                t = t[1:]
            while t.endswith(')'):
                cpbuf.append(')')
                t = t[:-1]
            yield t
            while cpbuf:
                yield cpbuf.pop()
    
    def load(self, s):
        self.input.extend(self.tokenize(s))
    
    def run(self):
        while self.input:
            self.step()
        return int(self.stack[-1])

    def step(self):
        try:
            t = self.input.popleft()
        except IndexError:
            return int(self.stack[-1])
        
        if t == '(':
            self.leftparen()
        
        elif t == ')':
            self.rightparen()
            
        elif t.isdigit():
            self.val(t)

        elif t in ['+', '*']:
            self.stack.append(t)

        else:
            raise ValueError("Unknown token: %s", t)
        
    def leftparen(self):
        self.stack.append('(')
    
    def rightparen(self):
        t = self.stack.pop()
        while t != '(':
            self.input.appendleft(t)
            t = self.stack.pop()

    def val(self, t):
        if not self.stack or self.stack[-1] == '(':
            self.stack.append(t)
            return
        
        op = self.stack.pop()
        if op.isdigit():
            raise ValueError("Two values without an intervening operator")
            
        if op == '+':
            self.add(t)
            return
        
        if op == '*':
            self.mul(t)
            return
        
        raise ValueError("Can't happen")

    def add(self, t):
        prev_val = self.stack.pop()
        if not (prev_val.isdigit()):
            raise ValueError("'+' preceded by non-value")
        self.stack.append(str(int(prev_val) + int(t)))
    
    def mul(self, t):
        prev_val = self.stack.pop()
        if not (prev_val.isdigit()):
            raise ValueError("'+' preceded by non-value")
        self.stack.append(str(int(prev_val) * int(t)))


In [11]:
c = Calculator()
part_1 = sum(c(x) for x in data)
part_1

6640667297513

In [50]:
OPERATORS = {
    '+': 0,
    '*': 0
}

In [51]:
def tokenize(s):
    rpbuf = []
    for t in s.split():
        while t.startswith('('):
            yield '('
            t = t[1:]
        while t.endswith(')'):
            rpbuf.append(')')
            t = t[:-1]
        if t:
            yield t
        while rpbuf:
            yield rpbuf.pop()

In [52]:
def shuntingyard(tokens):
    """Infix -> RPN"""
    stack = []
    for t in tokens:
        if t.isdigit():
            yield t
        elif t in OPERATORS:
            while stack and stack[-1] in OPERATORS and OPERATORS[stack[-1]] >= OPERATORS[t]:
                yield stack.pop()
            stack.append(t)
        elif t == '(':
            stack.append(t)
        elif t == ')':
            while stack and stack[-1] != '(':
                yield stack.pop()
            _ = stack.pop()
        else:
            raise ValueError("Unknown token: %s", t)
    while stack:
        yield stack.pop()     

In [53]:
def evalrpn(tokens):
    stack = []
    for t in tokens:
        if t.isdigit():
            stack.append(int(t))
        else:
            if t == '*':
                stack.append(stack.pop() * stack.pop())
            elif t == '+':
                stack.append(stack.pop() + stack.pop())
            else:
                raise ValueError('Undefined operator: %s' % t)
    return stack.pop()

In [54]:
def calculate(s):
    return evalrpn(shuntingyard(tokenize(s)))

In [55]:
s = '2 * 3 + (4 * 5)'

In [56]:
part_1 = sum(calculate(x) for x in data)
part_1

6640667297513

## part 2

In [57]:
OPERATORS = {
    '+': 1,
    '*': 0
}

In [58]:
part_2 = sum(calculate(x) for x in data)
part_2

451589894841552